In [1]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
import getpass
import time
import pandas as pd
import os
import snowflake.connector
from utils import snowflake_login, descargar_segmento
from snowflake.connector.pandas_tools import write_pandas
from datetime import datetime
import streamlit as st

In [2]:
user, cursor, snow = snowflake_login()

Intento 1
Error: 251005: User is empty
Incorrect Password - provide again
Intento 2
Correct Password - connected to SNOWFLAKE


In [3]:
f = datetime.now().strftime('%Y%m%d_%H%M%S')
f = "'" + f + "'"
f

"'20250723_182255'"

In [7]:
with open('Queries/DUPLICADOS.sql', 'r', encoding="utf8") as file: command = file.read()
command

'WITH PP AS (\nSELECT\n    DISTINCT A.CODIGO_TIENDA, B.FAMILIA, COUNT(DISTINCT A.PVP_NUEVO) AS P\nFROM\n    SANDBOX_PLUS.DWH.INPUT_PRICING_ACUMULADO A\nLEFT JOIN\n    SANDBOX_PLUS.DWH.RESULTADO_PRICING_HISTORICO_BIS B\nON\n    A.ORIN = B.ORIN AND A.CODIGO_TIENDA = B.GEOG_LOCL_COD\nWHERE\n    FECHA_EJECUCION = (SELECT MAX(FECHA_EJECUCION) FROM SANDBOX_PLUS.DWH.RESULTADO_PRICING_HISTORICO_BIS)\nAND\n    TABLA={f}\nGROUP BY ALL\n)\n\nSELECT\n    A.*, B.FAMILIA, B.ARTC_ARTC_DESC, B.STCK_PRECIO_VENTA_DIA_CIVA\nFROM\n    SANDBOX_PLUS.DWH.INPUT_PRICING_ACUMULADO A\nJOIN\n    SANDBOX_PLUS.DWH.RESULTADO_PRICING_HISTORICO_BIS B\nON\n    A.ORIN = B.ORIN AND A.CODIGO_TIENDA = B.GEOG_LOCL_COD\nJOIN\n    PP\nON\n    A.CODIGO_TIENDA = PP.CODIGO_TIENDA AND B.FAMILIA = PP.FAMILIA AND P>1\nWHERE\n    FECHA_EJECUCION=(SELECT MAX(FECHA_EJECUCION) FROM SANDBOX_PLUS.DWH.RESULTADO_PRICING_HISTORICO_BIS)\nAND\n    TABLA={f}\nORDER BY\n    GEOG_LOCL_COD, B.FAMILIA;'

In [8]:
cursor.execute(command.format(f=f))

In [ ]:
d = cursor.fetch_pandas_all()
d

,CODIGO_TIENDA,ORIN,PVP_NUEVO,EFFECTIVE_DATE,TABLA,FAMILIA,ARTC_ARTC_DESC,STCK_PRECIO_VENTA_DIA_CIVA


In [5]:
c = "WHERE LGL.GEOG_LOCL_COD IN (101, 115) AND LAA.ORIN IN ('1000038127')"

In [12]:
with open('Queries/PRECIOS.sql', 'r', encoding="utf8") as file: command = file.read()
command

"WITH P AS (\nSELECT\n    ARTC_ARTC_ID, GEOG_LOCL_ID, MEDIAN(STCK_PRECIO_VENTA_DIA_CIVA) AS PRECIO_MEDIANO\nFROM\n    MSTRDB.DWH.FT_STOCK\nWHERE\n    TIEM_DIA_ID = CURRENT_DATE - 30\nGROUP BY ALL\n)\n\n, STOCK AS (\nSELECT\n    DISTINCT FS.ARTC_ARTC_ID, FS.GEOG_LOCL_ID, FS.STCK_PRECIO_VENTA_DIA_CIVA AS PVP_ACTUAL,\n    CASE\n        WHEN FCUR.ORIGIN_COUNTRY_ID = 'UYU' THEN FCUR.COSTO_UNITARIO_RMS\n        ELSE FS.STCK_COSTO_PROM_POND\n    END AS COSTO_UNITARIO,\n    COSTO_UNITARIO * ROUND(FS.STCK_PRECIO_VENTA_DIA_CIVA/FS.STCK_PRECIO_VENTA_DIA, 2) AS COSTO_UNITARIO_CON_IVA\nFROM\n    MSTRDB.DWH.FT_STOCK FS\nJOIN\n    MSTRDB.DWH.FT_COSTO_UNITARIO_RMS FCUR\nON\n    FS.GEOG_LOCL_ID = FCUR.GEOG_LOCL_ID AND FS.ARTC_ARTC_ID = FCUR.ARTC_ARTC_ID\n    AND FCUR.TIEM_DIA_ID = (SELECT MAX(TIEM_DIA_ID) FROM MSTRDB.DWH.FT_COSTO_UNITARIO_RMS)\nWHERE\n    FS.TIEM_DIA_ID = CURRENT_DATE - 1\n)\n\nSELECT\n    LAA.ORIN, LGL.GEOG_LOCL_COD AS LOCAL, PVP_ACTUAL, PRECIO_MEDIANO, COSTO_UNITARIO_CON_IVA,\n    RO

In [15]:
cursor.execute(command.format(cond=c))

In [16]:
p = cursor.fetch_pandas_all()
p

,ORIN,LOCAL,PVP_ACTUAL,PRECIO_MEDIANO,COSTO_UNITARIO_CON_IVA,MARGEN
0,1000038127,101,245.0,244.0,156.404,0.36
1,1000038127,115,198.0,244.0,156.404,0.21


In [4]:
duplicados = descargar_segmento(cursor, 'DUPLICADOS', cond=f)
duplicados

,CODIGO_TIENDA,ORIN,PVP_NUEVO,EFFECTIVE_DATE,TABLA,FAMILIA,ARTC_ARTC_DESC,STCK_PRECIO_VENTA_DIA_CIVA


In [7]:
p = descargar_segmento(cursor, 'PRECIOS', cond=c)
p

,ORIN,LOCAL,PVP_ACTUAL,PRECIO_MEDIANO,COSTO_UNITARIO_CON_IVA,MARGEN
0,1000038127,101,245.0,244.0,156.404,0.36
1,1000038127,115,198.0,244.0,156.404,0.21
